In [1]:
import wisps
import splat
import wisps.simulations as wispsim
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
import scipy.integrate as integrate
from tqdm import tqdm
import numpy as np
import wisps.simulations as wispsim
import numba
import bisect

%matplotlib inline

In [2]:
lf=wisps.LUMINOSITY_FUCTION
lfdes=wisps.DES_LUMINOSITY_FUCTION
maglimits=wisps.MAG_LIMITS

In [3]:
obs=pd.read_csv(wisps.OUTPUT_FILES+'//observation_log_with_limit.csv')

In [4]:
ras=obs['ra (deg)']
decs=obs['dec(deg)']

In [5]:
direcs=SkyCoord(ra=ras*u.deg, dec=decs*u.deg)

In [6]:
n=2
zscl=280

In [7]:
import splat

In [8]:
import copy

In [9]:
import numpy

In [10]:
dmin=5
dmax=10000

In [12]:
from wisps.simulations import custom_volume_correction

In [13]:
juricp=wisps.OBSERVED_POINTINGS[0]

In [26]:
class Pointing(object):
    ## a pointing object making it easier to draw samples
    
    def __init__(self, **kwargs):
        #only input is the direction
        self.coord=kwargs.get('coord', None)
        self._samples={}
        self.survey=kwargs.get('survey', None)
        self.mag_limits=None
        self.dist_limits=None
        self.name=kwargs.get('name', None)
        self.volume=None
    
    def compute_distance_limits(self):
        """
        computes distance limits based on limiting mags
        """
        rels=wisps.POLYNOMIAL_RELATIONS
        spgrid=np.arange(20, 38)
        if self.mag_limits is None:
            pass
        else:
            #use F140W for 3d-hst pointing and f110w for wisps
            pol=None
            maglmts=None
            if self.survey=='wisps':
                pol=rels['sp_F110W']
                maglmts= self.mag_limits['F110W']
            if self.survey=='hst3d':
                pol=rels['sp_F140W']
                maglmts=self.mag_limits['F140W']

            #compute asbmags using abolute mag relations
            absmags=pol(spgrid)
            relfaintmags=np.array([maglmts[0] for s in spgrid])
            relbrightmags=np.array([maglmts[1] for s in spgrid])
            
            #compute distances
            dmins=get_distance(absmags, relbrightmags)
            dmaxs=get_distance(absmags, relfaintmags)

            distances=np.array([dmaxs, dmins]).T

            self.dist_limits=dict(zip(spgrid, distances))
            #create a dictionary

    def computer_volume(self):
        """
        given area calculate the volume
        """
        volumes={}
        solid_angle=SOLID_ANGLE
        for k in self.dist_limits.keys():
             vc=custom_volume_correction(self.coord,  self.dist_limits[k][1],self.dist_limits[k][0])
             volumes['vc_'+str(k)]=vc
             volumes[k]= vc*0.33333333333*(self.dist_limits[k][0]**3-self.dist_limits[k][1]**3)

        self.volume=volumes
    

In [27]:
def get_survey(pointing):
    if pointing.startswith('par'):
        return 'wisps'
    else:
        return 'hst3d'

In [28]:
from wisps.utils.tools import get_distance
from wisps.simulations import SOLID_ANGLE
import splat.simulate as spsim
from functools import partial
import bisect

In [29]:
ras=obs['ra (deg)']
decs=obs['dec(deg)']
surveys=obs.pointing.apply(get_survey)

In [30]:
def make_pointing(ra, dec, survey, name):
    coord=SkyCoord(ra=ra*u.deg,dec=dec*u.deg )
    return Pointing(coord=coord, survey=survey, name=name)
pnts=[make_pointing(ra, dec, survey, name) for ra, dec, survey, name in zip(ras, decs, surveys, obs.pointing.values)]

In [31]:
for pnt in tqdm(pnts):
    pnt.mag_limits=maglimits[pnt.survey]
    pnt.compute_distance_limits()
    pnt.computer_volume()
    #pnt.create_sample(nsample=1000)

100%|██████████| 533/533 [00:24<00:00, 21.76it/s]


In [32]:
vols=[x.volume for x in pnts]

In [33]:
import pickle
with open(wisps.OUTPUT_FILES+'/ldwarfs_scale_height.pkl', 'wb') as file:
    pickle.dump(vols,file)

In [35]:
#vols

In [ ]:
#float(integrate.trapz(rho*(d**2),x=d)/integrate.trapz(d**2,x=d))

In [ ]:
import wisps